In [1]:
import torch
from torch import nn
from tidybot2.utils import get_policy, rmat_to_quat, rot6d_to_rmat
from tidybot2.policy_wrapper import PolicyWrapper
import numpy as np

/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ckpt_path = '/juno/u/aadityap/universal_manipulation_interface/data/test/checkpoints/epoch_8450.ckpt'

In [3]:
policy = get_policy(ckpt_path)

You are using the CTM base workspace! Ensure that you don't wish to use the normal DP base workspace.

============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['arm_rot_axis_angle', 'gripper_pos', 'arm_rot_axis_angle_wrt_start', 'base_pose', 'arm_pos']
using obs modality: rgb with keys: ['base_image', 'wrist_image']
using obs modality: depth with keys: []
using obs modality: scan with keys: []


/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/juno/u/aadityap/miniforge3/envs/robodiff-retry/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Diffusion params: 6.721550e+07
Vision params: 2.239418e+07
_output_dir
global_step
epoch


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
policy = policy.to(device)
policy.eval()
for param in policy.parameters():
    param.requires_grad = False
pw = PolicyWrapper(policy, n_obs=2, n_acts=8, d_pos=6, d_rot=6, device=device)

In [10]:
# This is for the mobile base, fill in with your shapes
obs = {
    'base_pose': np.zeros(3),
    'arm_pos': np.zeros(3),
    'arm_rot_axis_angle': np.zeros(6),
    'arm_rot_axis_angle_wrt_start': np.zeros(6),
    'gripper_pos': np.zeros(1),
    'base_image': np.zeros((84, 84, 3), dtype=np.uint8),
    'wrist_image': np.zeros((84, 84, 3), dtype=np.uint8),
}

In [11]:
action = pw.get_action(obs)
print(action)

[-7.6663634e-04  2.2800708e-04  4.4526644e-03 -5.3826803e-03
  4.1822963e-03 -1.8360387e-03  9.5115089e-01 -2.3221029e-02
 -1.1508905e-02 -6.1398495e-02  9.6268797e-01  1.9202767e-02
  9.5908457e-01]


In [ ]:
base_pose = action[:3]
arm_pos = action[3:6]
arm_6d = torch.from_numpy(action[6:12])
arm_quat = rmat_to_quat(rot6d_to_rmat(arm_6d))
gripper_pos = action[12:13]

action_dict = {
    'base_pose': base_pose,
    'arm_pos': arm_pos,
    'arm_quat': arm_quat,
    'gripper_pos': gripper_pos
}